In [1]:
pip install -e ./cvaei


Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///Users/mayanknautiyal/Desktop/Codes/CVAEI/cvaei
ERROR: file:///Users/mayanknautiyal/Desktop/Codes/CVAEI/cvaei does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cvaei
import torch
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from cvaei.models.cvae_inference import CVAE 


In [3]:

input_dim = 10  
latent_dim = 2
conditional_dim = 5  
encoder_hidden_dims = [256, 128, 64]
decoder_hidden_dims = [64, 128, 256]


network = CVAE(input_dim=input_dim, latent_dim=latent_dim, output_dim=input_dim,
        conditional_dim=conditional_dim, encoder_hidden_dims=encoder_hidden_dims,
        decoder_hidden_dims=decoder_hidden_dims)


optimizer = optim.Adam(network.parameters(), lr=1e-3)


num_samples = 1000  
x_dummy = torch.randn(num_samples, input_dim)  
theta = torch.tensor(x_dummy, dtype=torch.float32)

c_dummy = torch.randn(num_samples, conditional_dim)
data = torch.tensor(c_dummy, dtype=torch.float32)

print(theta.shape, data.shape)

train_dataset = TensorDataset(data, theta)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)



network.train_model(train_loader=train_loader, optimizer=optimizer, epochs=10, device=device)



/var/folders/b1/bkdknbw50tv9cxhq0px4fp6h0000gn/T/ipykernel_9875/1302109949.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  theta = torch.tensor(x_dummy, dtype=torch.float32)
/var/folders/b1/bkdknbw50tv9cxhq0px4fp6h0000gn/T/ipykernel_9875/1302109949.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(c_dummy, dtype=torch.float32)


torch.Size([1000, 10]) torch.Size([1000, 5])
Epoch [1/10], Beta: 0.0000, Average Loss: 19.9848, Recon Loss: 9.9924, Misfit Loss: 9.9924, KL Div: 0.0235
Epoch [2/10], Beta: 0.1000, Average Loss: 19.3296, Recon Loss: 9.6071, Misfit Loss: 9.6071, KL Div: 1.1538
Epoch [3/10], Beta: 0.2000, Average Loss: 18.5618, Recon Loss: 9.0189, Misfit Loss: 9.0189, KL Div: 2.6199
Epoch [4/10], Beta: 0.3000, Average Loss: 18.1985, Recon Loss: 8.8256, Misfit Loss: 8.8256, KL Div: 1.8245
Epoch [5/10], Beta: 0.4000, Average Loss: 17.5075, Recon Loss: 8.2845, Misfit Loss: 8.2845, KL Div: 2.3465
Epoch [6/10], Beta: 0.5000, Average Loss: 17.3462, Recon Loss: 8.1492, Misfit Loss: 8.1492, KL Div: 2.0955
Epoch [7/10], Beta: 0.6000, Average Loss: 17.3769, Recon Loss: 8.0961, Misfit Loss: 8.0961, KL Div: 1.9745
Epoch [8/10], Beta: 0.7000, Average Loss: 17.5024, Recon Loss: 8.1254, Misfit Loss: 8.1254, KL Div: 1.7880
Epoch [9/10], Beta: 0.8000, Average Loss: 17.5798, Recon Loss: 8.0972, Misfit Loss: 8.0972, KL Div: